# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [17]:
import pandas as pd
from pymystem3 import Mystem 
m = Mystem()
Data = pd.read_csv('/datasets/data.csv')
Data.info() # посмотрим информацию 
Data.head() # визуально изучим таблицу 


 

0.40000000000000002220446049250313080847263336181641


0.4

**Вывод** Столбцы days_employed и total_income равны друг другу, но отличаются от остальных кол-вом элементов. Их нужно заполнить. С типами данных все в порядке, за исключением неудобного формата для total_income. Его мы округлим и переведем в int. Значения days_employed ещё и часто отрицательные. Также есть проблемы с регистром в education.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [16]:
income_grouped_mean = Data.groupby('income_type')['total_income'].mean()#среднее значение 
# print(income_grouped_mean)
income_grouped_median = Data.groupby('income_type')['total_income'].median()#медианное значение
#print(income_grouped_median) # данные среднего и медианы совпадают, заполняю пропуски
def fillbygroup(Data, row): # функция заменяет NaN в row на медианное значение этого столбца у соответствующего income_type
    unique_inc_type = Data['income_type'].unique()
    for type in unique_inc_type:
        Data.loc[Data['income_type'] == type, row] = Data.loc[Data['income_type'] == type, row].fillna(Data[Data['income_type'] == type]['total_income'].median())
    return Data
Data = fillbygroup(Data, 'total_income')
#Data.info() # снова вызываю метод и подтверждаю замену пропусков.
Data['days_employed'] = Data['days_employed'].apply(abs) # избавляюсь от отрицательных значений
Data = fillbygroup(Data, 'days_employed') # применяю функцию
dob_mean = Data['dob_years'].mean() # в возрасте нет выдающихся значений, поэтому заполняю нули средним возрастом
Data.loc[Data['dob_years'] == 0, 'dob_years'] = dob_mean
Data['dob_years'] = Data['dob_years'].round().astype('int')
#print(Data.info()) # пропусков больше нет
Data



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**Заполнил пропуски в столбце total_income на среднее значение по столбцу и заменил нулевой возраст в столбце dob_years на средний возраст , а так же избавился от отрицательных значений

### Замена типа данных

In [17]:
Data['days_employed'] = Data['days_employed'].round().astype('int') # округляю float значения в большую сторону, а затем перевожу в int
Data['total_income'] = Data['total_income'].round().astype('int')
Data['gender'].value_counts()
Data=Data.drop(Data[Data['gender']=='XNA'].index) #удаляю в столбце {'gender'} значение XNA
Data['gender'].value_counts()
Data['children'].value_counts()
Data['children']=Data['children'].replace(-1,1) # заменяю знечение в столбце ['children'] -1 на 1 и 20 на 2. 
Data['children']=Data['children'].replace(20,2) # скорее всего это были неправильно занесенные данные
Data['children'].value_counts() # Теперь данные корректны
Data['dob_years'].unique() 
Data.info() # total_income теперь int
Data


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224792,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,156000,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89673,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**В столбце ['total_incom'] тип  был поменян с вещественного на целочисленный.Удалены выделяющиеся объекты в  'gender', а в столбце 'children' значения -1 и 20 заменены на 1 и 2 соответственно. По моему мнению это человеческие ошбки.

 ### Обработка дубликатов

In [18]:
del Data['days_employed']# решил просто удалить, так как этот столбец не нужен для анализа
Data['education'] = Data['education'].str.lower()# тут только проблемы с регистром, исправляю
print(Data.duplicated().sum())# нахожу дубликаты 
Data['education'].value_counts()
purpose_counts = Data['purpose'].value_counts() 
print(purpose_counts) # тут придется выделять основу
Data = Data.drop_duplicates().reset_index()# 71 дубликат. Удаляю
print(Data.duplicated().sum())
Data.isna().sum()# Проверка на пропуски
Data

71
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием   

,index,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224792,операции с жильем
21449,21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,156000,сделка с автомобилем
21450,21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89673,недвижимость
21451,21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**В ходе обработки пропусков был удален столбец по стажу работы, т.к он не нужен для анализа.Мы не можем получить недостающие данные о трудовом стаже и доходе. Эти значения можем удалить,а столбец ['days_employed'] убрать вовсе, чтобы не портили таблицу.
Было замечено, что значения пропущены как в столбце ['days_employed'] так и в столбце ['total_income'], причем с одинаковым индексом. Возможные причины появления пропусков связяны с тем, что клиент не помнит свой трудовой стаж или он просто не работает, а про тип занятости мог соврать. Скорее всего эти пропуски не связяны с техническими проблемами, а человеческий фактор. 
Пропуски по доходу были заменены на средние значения дохода по соответствующему роду деятельности.Дубликаты были  обнаружены в столбце ['education'] и отличались они только регистром. Поэтому воспользовался функцией str.lower() и понизил весь текст по регистру. После этого ушли все дубликаты в столбце ['education'].

### Лемматизация

In [19]:
Data['purpose'].value_counts() # уникальные цели 
#В функцию lemmatization аргументом попадает цель purpose. 
# Группа условий проверяет lemma_tmp и 
#определяет ее в одну из четырех: жилье, автомобиль, образование, личные цели.
def lemmatizer(Data): # функция лемматизирует каждую цель, затем по ключевому слову заменяет цель в таблице
    for i in range(0, len(Data['purpose'])):
        lemmas = m.lemmatize(Data.loc[i,'purpose'])
        if 'свадьба' in lemmas:
            Data.loc[i,'purpose'] = 'свадьба'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            Data.loc[i,'purpose'] = 'недвижимость'
        if 'автомобиль' in lemmas:
            Data.loc[i,'purpose'] = 'автомобиль'
        if 'образование' in lemmas:
            Data.loc[i,'purpose'] = 'образование'
    return Data

Data = lemmatizer(Data)
Data.head(5)



,index,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,недвижимость
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,недвижимость
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,образование
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


**Вывод**После лемматизации столбец выглядит понятнее, видно 4 категории целей, которыу можно удобно группировать и исследовать

### Категоризация данных

In [20]:
# Функция для разбивки на категории в зависимости от возраста.
def dob_years_group(age):
    if 19<=age<=35:
        return 'молодые'
    if 36<age<59:
        return 'взрослые'
    return 'пенсионеры'
dob_years_group(60)
dob_years_group=Data['dob_years'].apply(dob_years_group)
Data['dob_years']=dob_years_group

# Функция для категоризации по уровню дохода
def total_income_group(income):
    if 20000<=income<70000:
        return "низкий доход"
    if 70001<=income<120000:
        return 'средний доход'
    if 120001<=income<200000:
        return 'высокий доход'
    return 'очень высокий доход'
total_income_group(70000)
total_income_gr=Data['total_income'].apply(total_income_group)
Data['total_income']=total_income_gr
Data.head(5)

,index,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,очень высокий доход,недвижимость
1,1,1,пенсионеры,среднее,1,женат / замужем,0,F,сотрудник,0,средний доход,автомобиль
2,2,0,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,высокий доход,недвижимость
3,3,3,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,очень высокий доход,образование
4,4,0,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,высокий доход,свадьба


**Вывод**В процессе категоризации были выбраны две группы, которые я разбивал на категории. Это Возраст и Доход. Категоризация проходила с помощью функиции с группой условий и метода apply. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
chil_debt=Data.groupby('children').agg({'debt':['count','sum']}) # агрегированная функция
chil_debt
chil_quality=chil_debt['debt']['sum']/chil_debt['debt']['count']
chil_quality # Подсчет доли

children
0    0.075444
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
dtype: float64

**Вывод**Из этих данных видно, что возврат кредит связан с количеством детей. Больше всего просрочек у людей с 1, 2,3,4 детьми. Нет росрочек в тех у кого 5 детей. У кого детей вообще нет- просрочек меньше. Скорее всего это связано с финансовой нагрузкой

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
family_grouped=Data.groupby('family_status').agg({'debt':['count','sum']}) 
family_debt_quality=family_grouped['debt']['sum']/family_grouped['debt']['count']
family_debt_quality

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093494
женат / замужем          0.075452
dtype: float64

**Вывод** Доля просрочек на всех "вдовцов/вдов" составляет- 0.06. Похожие результаты можно увидеть и у людей, которые в "разводе". Доля просрочек на всех разведенных составляет - 0.07. Я это могу объяснить тем, что человек, который был в официальном браке, а в дальнейшем развелся или потерял мужа/жену, инеет высокий уровень ответственности.
Доля просрочек на всех женатых/ замужем составляет- 0.075. Можно сказать, что доля просрочек у "женат / замужем" и в "разводе" почти одинакова.
Что касается людей в "гражданском браке" и "не женатых / не замужем", то тут самый плохой показатель. Хуже всего этот показатель у "не женат / не замужем"- доля просрочек - 0.1. Думаю, что это связано с нежеланием брать ответственность. !

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
total_income_grouped=Data.groupby('total_income').agg({'debt':['sum','count']})
total_income_quality=total_income_grouped['debt']['sum']/total_income_grouped['debt']['count']
total_income_grouped['debt']
total_income_quality
Data_pivot=Data.pivot_table(index=['dob_years'],columns='total_income',values='debt',aggfunc='sum')
Data_pivot#Сводная таблица просрочкам кредита в зависимости от возраста и уровня дохода. 

total_income,высокий доход,низкий доход,очень высокий доход,средний доход
dob_years,,,,
взрослые,368,41,192,246
молодые,348,35,133,182
пенсионеры,81,25,33,57


**Вывод** Лушчий средний показатель имеют клиенты с низким доходом. Рассуждая логически, клиенты с низким доходом понимают, что в им нужна положительная кредитная история, для того, чтобы одобрили следующий. 

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
purpose_grouped=Data.groupby('purpose').agg({'debt':['sum','count']})
purpose_convers=purpose_grouped['debt']['sum']/purpose_grouped['debt']['count']
purpose_convers

purpose
автомобиль      0.093590
недвижимость    0.072340
образование     0.092200
свадьба         0.080034
dtype: float64

**Вывод** Больше всего просрочек у кредитов на автомобиль и образование. Доля их получается 0.092. Можно это объяснить тем, что на рынке автокредитования создаются очень привлекательные предложения и человек порой берет кредит не оценевая аддекватно свои возможности.
Кредит на личные цели и кредит на недвижимось имеют меньше просрочек. Человек, выбирая ипотеку, хорошо просчитывает свои возможности, так как срок платежа очень большой.

## Шаг 4. Общий вывод

Из полученных данных я могу сделать вывод, что самыми надежными заемщиками являются люди, которые берут ипотеку, с низким или очень высоким уровнем заработка, имеющие детей, состоящие в официальном браке(лучше вдовец или вдова)